In [1]:
# Import Libraries
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D

In [2]:
# Load Datasets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape) # 4차원 Tensor
print(X_test.shape)

170498071/170498071 [==============================] - 12s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [3]:
# min-max scaling
# RGB : [0, 255]
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.

In [4]:
model = Sequential([
                    Conv2D(input_shape=(32, 32, 3), filters=6, kernel_size=(3, 3),
                           strides=(1, 1), padding="same", activation='relu'),
                    Conv2D(16, 3, 1, padding="same", activation='relu'),
                    MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
                    Conv2D(24, 3, 1, padding="same", activation='relu'),
                    Conv2D(48, 3, 1, padding="same", activation='relu'),
                    MaxPool2D(), 
                    Flatten(), # activation map : 48 x 8 x 8
                    Dense(1024, activation='relu'), # fully-connected
                    Dense(10, activation="softmax") # output-class 10
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 6)         168       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 16)        880       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 16)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 24)        3480      
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 48)        10416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 48)         0         
 2D)                                                    

In [5]:
# training setup
from tensorflow.keras.optimizers import SGD

## config variables
batch_size = 128
lr = 0.01 # 1e-5
epochs = 15

# Set optimizer, loss function, metrics, callback function
optimizer = SGD(lr, momentum=0.9)
loss_fn = "sparse_categorical_crossentropy" # multi-class classification
metrics = ["accuracy"]

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

In [ ]:
# model training
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test), # 매 epoch마다 validation 성능 체크 가능!
                    verbose=1)

Epoch 1/15
391/391 [==============================] - 51s 130ms/step - loss: 1.7871 - accuracy: 0.3566 - val_loss: 1.4207 - val_accuracy: 0.4883
Epoch 2/15
391/391 [==============================] - 47s 119ms/step - loss: 1.3311 - accuracy: 0.5279 - val_loss: 1.2134 - val_accuracy: 0.5679
Epoch 3/15
287/391 [=====================>........] - ETA: 11s - loss: 1.1173 - accuracy: 0.6052

In [ ]:
# evaluate
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Loss : %.4f, Accuracy : %.4f" % (loss, acc))

In [ ]:
## loss visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.subplot(1, 2,1)
plt.plot(history.history['loss'],'b-', label = "training")
plt.plot(history.history['val_loss'], 'r:', label = "validation")
plt.title("model - loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.title("model - accuracy")

plt.plot(history.history['accuracy'], 'b-', label = "training")
plt.plot(history.history['val_accuracy'], 'r:', label = "validation")

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# load ResNet50
from tensorflow.keras.applications import ResNet50   # pretrained model -> ImageNet

resnet = ResNet50(
    include_top=True,  # classification : True,  embedding : False
    weights=None,
    input_shape=(32, 32, 3),
    pooling="avg",
    classes=10,
    classifier_activation="softmax"
)

In [ ]:
# see ResNet50 model
resnet.summary()

In [ ]:
### resnet training strategy

batch_size = 32
epochs = 10
lr = 0.1

optimizer = SGD(lr, momentum=0.9)
loss_fn = "sparse_categorical_crossentropy"
metrics = ["accuracy"]

resnet.compile(optimizer=optimizer,
               loss=loss_fn,
               metrics=metrics)

In [ ]:
# training ResNet50
resnet_history = resnet.fit(X_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(X_test, y_test),
                            verbose=1)

In [ ]:
## loss visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.subplot(1, 2,1)
plt.plot(resnet_history.history['loss'],'b-', label = "training")
plt.plot(resnet_history.history['val_loss'], 'r:', label = "validation")
plt.title("model - loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.title("model - accuracy")

plt.plot(resnet_history.history['accuracy'], 'b-', label = "training")
plt.plot(resnet_history.history['val_accuracy'], 'r:', label = "validation")

plt.legend()
plt.tight_layout()
plt.show()